# Correlating Returns

In [8]:
import os
import pandas as pd
from nltk.corpus import reuters
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [9]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [10]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                      
                             open      high     low     close    volume
time                                                                   
2021-04-05 00:00:00-04:00  123.87  126.1601  123.07  125.9001  79312943
2021-04-06 00:00:00-04:00  126.50  127.1300  125.65  126.2300  70316590
2021-04-07 00:00:00-04:00  125.83  127.9200  125.14  127.9100  75433362
2021-04-08 00:00:00-04:00  128.95  130.3900  128.52  130.3700  80194739
2021-04-09 00:00:00-04:00  129.80  133.0400  129.47  133.0000  94171787

In [11]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-04-05,125.9001
2021-04-06,126.2300
2021-04-07,127.9100
2021-04-08,130.3700
2021-04-09,133.0000


In [12]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2021-04-06,0.002620
2021-04-07,0.013309
2021-04-08,0.019232
2021-04-09,0.020173
2021-04-12,-0.013083


In [25]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [46]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2021-05-03 00:00:00
retrieving news from: 2021-05-02 00:00:00
retrieving news from: 2021-05-01 00:00:00
retrieving news from: 2021-04-30 00:00:00
retrieving news from: 2021-04-29 00:00:00
retrieving news from: 2021-04-28 00:00:00
retrieving news from: 2021-04-27 00:00:00
retrieving news from: 2021-04-26 00:00:00
retrieving news from: 2021-04-25 00:00:00
retrieving news from: 2021-04-24 00:00:00
retrieving news from: 2021-04-23 00:00:00
retrieving news from: 2021-04-22 00:00:00
retrieving news from: 2021-04-21 00:00:00
retrieving news from: 2021-04-20 00:00:00
retrieving news from: 2021-04-19 00:00:00
retrieving news from: 2021-04-18 00:00:00
retrieving news from: 2021-04-17 00:00:00
retrieving news from: 2021-04-16 00:00:00
retrieving news from: 2021-04-15 00:00:00
retrieving news from: 2021-04-14 00:00:00
retrieving news from: 2021-04-13 00:00:00
retrieving news from: 2021-04-12 00:00:00
retrieving news f

In [47]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2021-05-03 00:00:00
retrieving news from: 2021-05-02 00:00:00
retrieving news from: 2021-05-01 00:00:00
retrieving news from: 2021-04-30 00:00:00
retrieving news from: 2021-04-29 00:00:00
retrieving news from: 2021-04-28 00:00:00
retrieving news from: 2021-04-27 00:00:00
retrieving news from: 2021-04-26 00:00:00
retrieving news from: 2021-04-25 00:00:00
retrieving news from: 2021-04-24 00:00:00
retrieving news from: 2021-04-23 00:00:00
retrieving news from: 2021-04-22 00:00:00
retrieving news from: 2021-04-21 00:00:00
retrieving news from: 2021-04-20 00:00:00
retrieving news from: 2021-04-19 00:00:00
retrieving news from: 2021-04-18 00:00:00
retrieving news from: 2021-04-17 00:00:00
retrieving news from: 2021-04-16 00:00:00
retrieving news from: 2021-04-15 00:00:00
retrieving news from: 2021-04-14 00:00:00
retrieving news from: 2021-04-13 00:00:00
retrieving news from: 2021-04-12 00:00:00
retrieving news f

In [48]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2021-05-03 00:00:00
retrieving news from: 2021-05-02 00:00:00
retrieving news from: 2021-05-01 00:00:00
retrieving news from: 2021-04-30 00:00:00
retrieving news from: 2021-04-29 00:00:00
retrieving news from: 2021-04-28 00:00:00
retrieving news from: 2021-04-27 00:00:00
retrieving news from: 2021-04-26 00:00:00
retrieving news from: 2021-04-25 00:00:00
retrieving news from: 2021-04-24 00:00:00
retrieving news from: 2021-04-23 00:00:00
retrieving news from: 2021-04-22 00:00:00
retrieving news from: 2021-04-21 00:00:00
retrieving news from: 2021-04-20 00:00:00
retrieving news from: 2021-04-19 00:00:00
retrieving news from: 2021-04-18 00:00:00
retrieving news from: 2021-04-17 00:00:00
retrieving news from: 2021-04-16 00:00:00
retrieving news from: 2021-04-15 00:00:00
retrieving news from: 2021-04-14 00:00:00
retrieving news from: 2021-04-13 00:00:00
retrieving news from: 2021-04-12 00:00:00
retrieving news

In [52]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [53]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [54]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)

In [55]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
    }
)

In [56]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [57]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")


# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg
2021-04-06,0.002620,0.262250,0.083030,0.141055
2021-04-07,0.013309,-0.002540,0.012705,0.130105
2021-04-08,0.019232,0.000585,-0.068855,0.114510
2021-04-09,0.020173,-0.025520,-0.052630,-0.013170
2021-04-12,-0.013083,0.062590,0.052780,0.131135
2021-04-13,0.023998,0.058765,0.057265,0.033165
2021-04-14,-0.017037,-0.050860,0.019220,0.133885
2021-04-15,0.018014,0.133615,-0.119125,0.082985
2021-04-16,-0.002602,0.186835,-0.118765,-0.006515
2021-04-19,0.005747,-0.000670,0.079920,-0.023520


In [58]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg
close,1.000000,-0.039457,-0.315832,-0.001966
aapl_avg,-0.039457,1.000000,-0.095380,0.065975
trade_avg,-0.315832,-0.095380,1.000000,-0.119020
economy_avg,-0.001966,0.065975,-0.119020,1.000000
